# About the Dataset

This dataset is about the E-commerce store sales, it can befound on Kaggle - https://www.kaggle.com/datasets/anandku79/kpidashboard

I will use Python to explore the data, and calculate real life business KPIs.

Then I will create dynamic dashboards using Tableau to show these KPIs and analyse them.

# Imports

In [87]:
import numpy as np 
import pandas as pd 
from datetime import datetime
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import zscore

# Exploratory Data Analysis 
Let's break it down and explore the data structure:

In [88]:
df = pd.read_csv('superstore.csv')
display(df.head(1))
print('There are',len(df),'rows, and', len(df.columns), 'columns in our data')

,Unnamed: 0,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Delivery Days,order year,order month
0,0,CA-2014-AB10015140-41954,11/11/2014,13-11-2014,First Class,AB-100151402,Aaron Bergman,Consumer,Oklahoma City,Oklahoma,...,Samsung Convoy 3,221,2,0.0,62,40,High,2,2014,11


There are 51290 rows, and 26 columns in our data


It looks like each row in the data is an order of a customer.

Let's check the data type and the number of null values of each column:

In [89]:
display(df.info())
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Region          51290 non-null  object 
 12  Market          51290 non-null  object 
 13  Product ID      51290 non-null  object 
 14  Category        51290 non-null  object 
 15  Sub-Category    51290 non-null  object 
 16  Product Name    51290 non-null  object 
 17  Sales           51290 non-null 

None

Index(['Unnamed: 0', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Region', 'Market', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Shipping Cost', 'Order Priority', 'Delivery Days', 'order year',
       'order month'],
      dtype='object')


There are no null values in the DF, now let's check for duplicated rows in the df:

In [90]:
df.duplicated().any()

False

Great! there are no duplications, seems like the data in clean already.

Let's check the first column - "Unamed: 0", I'll check what values does it hold:

In [91]:
first_column = list(df.columns)[0]
df[first_column].head()

0    0
1    1
2    2
3    3
4    4
Name: Unnamed: 0, dtype: int64

Seems like an index, I will validate it by checking if the length of the total df equal to the number of unique values in this column (meaning each value in this column appears only once):

In [92]:
len(df) == len(df[list(df.columns)[0]])

True

Now that I validated it is just an index column, I will remove it since it is not helpful to my analysis:

In [93]:
df = df.drop(columns=['Unnamed: 0'])

Let's move on to the next column - 'Order ID', first of all I want to check if the values in this column are unique. I can check how many rows belong to each Order ID:

In [94]:
grouped_df = df.groupby('Order ID').size().reset_index(name='count').sort_values('count', ascending = False)
grouped_df

,Order ID,count
4604,CA-2015-SV20365140-42268,14
12570,IN-2013-TB21055113-41562,13
20624,MX-2015-PO1885082-42279,13
22441,TO-2015-AB600131-42299,13
13269,IN-2014-MH1778527-41697,13
...,...,...
15904,IT-2014-MY17380104-41983,1
15905,IT-2014-MY1738045-41950,1
6495,ES-2013-MG17695120-41566,1
6494,ES-2013-MG176808-41611,1


It seems like customers can order several items in each order, let's check how many customers order more than one item in a order:

In [95]:
percentage = round(len(grouped_df[grouped_df['count']>1])/len(grouped_df)*100,2)
print(str(percentage)+'%','out of total orders have more than one item')

49.72% out of total orders have more than one item


# KPI Analysis
In this section I will calculte several business KPIs from the data, I will zoom in and check each one in a monthly prespective and then I will uncover interesting insights about it:

1. Total Sales (monthly sum of saels), this KPI is crucial as it represents the overall financial health of the supermarket, indicating the total earnings from sales.


In [96]:
#Sum of sales per month and year
total_sales_df = df.groupby(['order year', 'order month'])['Sales'].sum().reset_index(name='total_sales').sort_values(['order year', 'order month'], ascending=True)

# Select the desired columns
KPI_df = total_sales_df[['order year', 'order month', 'total_sales']]

KPI_df.head()

,order year,order month,total_sales
0,2012,1,98666
1,2012,2,103493
2,2012,3,135475
3,2012,4,114048
4,2012,5,157910


2. **Average Revenue Per Customer** (monthly sum of sales divided by number of customers each month), this KPI reveals the average amount a customer spends per visit helping to evaluate sales effectiveness and customer behavior.


In [97]:
#Sum of costumers per month and year
count_of_customers_df = df.groupby(['order year', 'order month'])['Customer ID'].count().reset_index(name='num_customers').sort_values(['order year', 'order month'], ascending=True)

#Merge the two data frames
average_revenue_per_customer_df = pd.merge(total_sales_df, count_of_customers_df, on=['order year', 'order month'])

# Calculate average sales per customer by dividing 'sum_sales' by 'num_customers'
KPI_df['average_revenue_per_customer_df'] = round(average_revenue_per_customer_df['total_sales'] / average_revenue_per_customer_df['num_customers'],2)

KPI_df.head()

,order year,order month,total_sales,average_revenue_per_customer_df
0,2012,1,98666,227.87
1,2012,2,103493,240.12
2,2012,3,135475,265.12
3,2012,4,114048,211.59
4,2012,5,157910,260.58


3. **Customer Retention Rate** (number of unique customers of a specific period dvided by the number of customers in the previous period), this KPI measures customer loyalty and satisfaction, helping to identify areas for improvement and customer relationship management.


In [98]:
# Count unique purchasing customers in each group
customer_counts = df.groupby(['order year', 'order month'])['Customer ID'].nunique()

# Shift the customer counts by one month
previous_month_customers = customer_counts.shift(1)

# Calculate retention rate
retention_rate = customer_counts / previous_month_customers

# Reset index for better representation
retention_rate = round(retention_rate.reset_index(),2)

retention_df = pd.merge(customer_counts, retention_rate, on=['order year', 'order month'])

retention_df.rename(columns={retention_df.columns[2]: 'Number of Customers', retention_df.columns[3]: 'Retention Rate'}, inplace=True)

KPI_df['retention_rate']= retention_df['Retention Rate']

KPI_df.head()

,order year,order month,total_sales,average_revenue_per_customer_df,retention_rate
0,2012,1,98666,227.87,NaN
1,2012,2,103493,240.12,0.94
2,2012,3,135475,265.12,1.32
3,2012,4,114048,211.59,0.96
4,2012,5,157910,260.58,1.23


# Prepare Data for Viz
In order to present the data in a Tableau dashboard, I will merge the two date columns into a single mm/dd/yyyy column (the day aspect will not show on the dashboard)

In [99]:
# Convert 'order month' and 'order year' to integers
KPI_df['order month'] = KPI_df['order month'].astype(int)
KPI_df['order year'] = KPI_df['order year'].astype(int)

KPI_df['Date'] = KPI_df.apply(lambda row: f"{int(row['order month']):02d}/01/{int(row['order year'])}", axis=1)

# Drop the original 'order year' and 'order month' columns
KPI_df.drop(['order year', 'order month'], axis=1, inplace=True)

# Move the 'Date' column to the beginning
date_column = KPI_df.pop('Date')
KPI_df.insert(0, 'Date', date_column)

KPI_df.head()

,Date,total_sales,average_revenue_per_customer_df,retention_rate
0,01/01/2012,98666,227.87,NaN
1,02/01/2012,103493,240.12,0.94
2,03/01/2012,135475,265.12,1.32
3,04/01/2012,114048,211.59,0.96
4,05/01/2012,157910,260.58,1.23


In [100]:
# Specify the path where you want to save the Excel file
excel_file_path = 'Data_for_dashboards.xlsx'

# Export the DataFrame to an Excel file
KPI_df.to_excel(excel_file_path, index=False)  

In [101]:
%%HTML
<div class='tableauPlaceholder' id='viz1693037838286' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Su&#47;SuperMarketKPIAnalysis&#47;TotalSalesDashboard&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SuperMarketKPIAnalysis&#47;TotalSalesDashboard' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Su&#47;SuperMarketKPIAnalysis&#47;TotalSalesDashboard&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1693037838286');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Using the dashboard in Tableau I can see a seasonal trend- the total sales are dropping significantly after December, I will dive deeper and try to discover why this is happening.

First I'll check what are the categories that provide the most incomes in December and how they behave on January:

In [102]:
total_sales_check_df = df.groupby(['order month','Category','Sub-Category'])['Sales'].sum().reset_index(name='total_income').sort_values(['total_income'], ascending=False)

total_sales_december = total_sales_check_df[total_sales_check_df['order month'] == 12].reset_index(drop=True)

display(total_sales_december.head())

total_sales_january = total_sales_check_df[total_sales_check_df['order month'] == 1].reset_index(drop=True)

display(total_sales_january.head())


,order month,Category,Sub-Category,total_income
0,12,Technology,Phones,213020
1,12,Furniture,Chairs,192563
2,12,Technology,Copiers,174713
3,12,Furniture,Bookcases,170922
4,12,Office Supplies,Storage,133101


,order month,Category,Sub-Category,total_income
0,1,Furniture,Chairs,78793
1,1,Furniture,Bookcases,77868
2,1,Technology,Phones,75984
3,1,Technology,Copiers,75624
4,1,Office Supplies,Storage,70205


It is clear to see that the top categories double to tripple their value on December compared to January.

A possible explanation can be the **Holiday Shopping Season**: December includes several major holidays such as Christmas and New Year's Eve, which are associated with gift-giving and increased consumer spending. During this time, people are more inclined to shop for presents, decorations, and holiday-related items, leading to a spike in sales.

## Statistical Testing - Discount Rates
Another reason for the rise in sales can be due to high discount rates in December compared to January, to determine if the average discounts in December are significantly higher than in January, I will use a statistical hypothesis test named t-test. This test compares the means of two independent groups to determine if the observed difference is statistically significant.

**Null Hypothesis (H0)**: The average discounts in December are not significantly different from those in January.

**Alternative Hypothesis (H1)**: The average discounts in December are significantly higher than those in January.

In [103]:
december_data = df[df["order month"] == 12]["Discount"]
january_data = df[df["order month"] == 1]["Discount"]

alpha = 0.05

# Calculate means
mean_december = december_data.mean()
mean_january = january_data.mean()

# Perform t-test
t_statistic, p_value = stats.ttest_ind(december_data, january_data)

# Print means and p-value
print(f"Mean Discount in December: {round(mean_december,2)}%")
print(f"Mean Discount in January: {round(mean_january,2)}%")
print(f"P-Value: {round(p_value,2)}")

if p_value < alpha:
    print("We reject the null hypothesis.")
    print("The average discounts in December are significantly higher than in January.")
else:
    print("We fail to reject the null hypothesis.")
    print("There is no significant difference in average discounts between December and January.")


Mean Discount in December: 0.15%
Mean Discount in January: 0.14%
P-Value: 0.22
We fail to reject the null hypothesis.
There is no significant difference in average discounts between December and January.


This means that the mean of the discounts are not significally different, however it is possible that the average discount remains the same in both months but the number of discounts offered is higher on December.

 To check that I will be comparing the proportions of orders with and without discounts between the months of December and January. 
 
 I will use a statistical hypothesis test named chi-square test, this test determines if the differences in proportions are statistically significant or if they could have occurred due to random chance. 

**Null Hypothesis (H0)**: There is no significant difference in the proportions of orders with and without discounts between December and January.

**Alternative Hypothesis (H1)**: There is a significant difference in the proportions of orders with and without discounts between December and January.

In [104]:
december_data = df[df["order month"] == 12]["Discount"].reset_index(drop=True)
january_data = df[df["order month"] == 1]["Discount"].reset_index(drop=True)

# Create binary columns indicating if the order had a discount or not
december_data_with_discount = (december_data > 0).astype(int)
january_data_with_discount = (january_data > 0).astype(int)

# Perform Chi-squared test
contingency_table = pd.crosstab(december_data_with_discount, january_data_with_discount)
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Map month numbers to month names
month_names = {0: "January", 1: "December"}
contingency_table.index = contingency_table.index.map(month_names)

# Calculate proportions (normalize=True)
proportions = contingency_table.div(contingency_table.sum(1), axis=0).round(3)

# Print proportions and p-value
print("Proportions of each group:")
proportions.rename(columns={proportions.columns[0]: 'p_of_non_discounted_orders', proportions.columns[1]: 'p_of_discounted_orders'}, inplace=True)
proportions = proportions.rename_axis('Month')

display(proportions)
print("P-Value:", round(p_value, 3))

if p_value < alpha:
    print("We reject the null hypothesis.")
    print("The proportion of orders with discounts in December is significantly different from that in January.")
else:
    print("We fail to reject the null hypothesis.")
    print("There is no significant difference in proportions of orders with discounts between December and January.")

Proportions of each group:


Discount,p_of_non_discounted_orders,p_of_discounted_orders
Month,,
January,0.601,0.399
December,0.588,0.412


P-Value: 0.533
We fail to reject the null hypothesis.
There is no significant difference in proportions of orders with discounts between December and January.


## Average Revenue Per Customer
I will examine the average revenue per customer, recognizing it as a composite metric that can be influenced by its individual components. While the overall average may appear stable, variations in the underlying values can still occur. This metric is composed of the total sales divided by the total number of customers, which adds a layer of complexity to its interpretation.
First I will check the average revenue of each of the customer segments: 

In [105]:
display(df.groupby(['Segment'])['Sales'].mean().round(2).reset_index(name='Average Revenue').sort_values(['Average Revenue'], ascending=False))

,Segment,Average Revenue
1,Corporate,247.38
2,Home Office,246.71
0,Consumer,244.90


When looking at the total numbers it seems like the value simillar in each segment and almost identical.

However, I will identify cases where the average sales amount remains steady but there is a sudden increase or decrease in the total number of customers or the total sales amount.

I will use a statistical hypothesis test named z-score test to assess the significance of these changes. By calculating the z-score, I can determine whether a particular shift in customer behavior is statistically significant or simply due to random variation.

The findings from this analysis can help identify different customer segments behaviors, for instance a certain group of customers might spend more than others in these cases:

In [106]:
average_revenue_per_customer_df['Average_Sales'] = round(average_revenue_per_customer_df['total_sales'] / average_revenue_per_customer_df['num_customers'],2)

# Calculate z-scores for total_sales and num_customers
z_scores_total_sales = zscore(average_revenue_per_customer_df['total_sales'])
z_scores_num_customers = zscore(average_revenue_per_customer_df['num_customers'])

# Define the threshold for significant change (in this case I used 2 standard deviations)
threshold = 2

# Filter rows where Average_Sales is similar and z-score exceeds the threshold for either total_sales or num_customers
similar_rows = average_revenue_per_customer_df[
    (np.abs(z_scores_total_sales) > threshold) | 
    (np.abs(z_scores_num_customers) > threshold)
]

print(similar_rows)


    order year  order month  total_sales  num_customers  Average_Sales
44        2015            9       480104           2018         237.91
46        2015           11       554150           2147         258.10
47        2015           12       502013           2153         233.17


I will create a viz showing the Average Revenue per Customer across various customer segments in this timeframe (September to Decmeber 2015):

In [107]:

%%HTML
<div class='tableauPlaceholder' id='viz1693047104649' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ec&#47;EcommereceFullDataDashboards&#47;CustomerSegemtAvgSalesinSeptembertoDecember2015&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='EcommereceFullDataDashboards&#47;CustomerSegemtAvgSalesinSeptembertoDecember2015' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ec&#47;EcommereceFullDataDashboards&#47;CustomerSegemtAvgSalesinSeptembertoDecember2015&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1693047104649');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

This is a valueable insight since now we know which customer segement is more likley to spend in this period of time and will be avle to personally target these customers and increase the store sales.

## Profit
Now I will examine the profit data of the Ecommerce store, first I will check the profit per Sub-Category to check if there are none profitable sub-categories:

In [108]:
%%HTML
<div class='tableauPlaceholder' id='viz1693296786934' style='position: relative'><noscript><a href='#'><img alt='Profit by Sub-Categories ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ec&#47;EcommereceFullDataDashboards&#47;ProfitbySub-Categories&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='EcommereceFullDataDashboards&#47;ProfitbySub-Categories' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ec&#47;EcommereceFullDataDashboards&#47;ProfitbySub-Categories&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1693296786934');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

It appears that the Tables sub-category is the only one not yielding profits. Nonetheless, certain stores might choose to sell products at a loss, as this could draw in additional potential customers who may buy more items, ultimately leading to a profitable overall transaction. Let's examine this hypothesis and check how what is the percentage of profitable orders that contain at least one table:

In [109]:
#Create a list of all orders that have Tables in them
tables_orders_lst = df['Order ID'][df['Sub-Category']=='Tables'].values.tolist()
tables_orders_df = df[df['Order ID'].isin(tables_orders_lst)]

#An order can have more than one product, we need to check profit gained from the total order
grouped_table_df = tables_orders_df.groupby('Order ID').agg({'Profit': 'sum', 'Sub-Category': 'count'}).reset_index()
grouped_table_df.rename(columns={'Sub-Category': 'Number_of_items'}, inplace=True)
display(grouped_table_df.head())
Total_tables_orders = grouped_table_df.shape[0]
Profitable_tables_orders = grouped_table_df[grouped_table_df['Profit']>0].shape[0]
profitable_tables_orders_percentage = round((Profitable_tables_orders/Total_tables_orders)*100,2)
print(f'The overall percentage of profitable orders that include at least one table as a product is: {profitable_tables_orders_percentage}%')

,Order ID,Profit,Number_of_items
0,AG-2013-LM70653-41352,172,5
1,AJ-2014-JL58359-41662,989,2
2,AO-2015-BD17254-42152,525,1
3,AO-2015-RM93754-42348,159,2
4,AU-2012-AG6758-41153,341,1


The overall percentage of profitable orders that include at least one table as a product is: 49.58%


In other words, more then half of the orders containing tables result in a negetive financial outcome, this means that this section of items should be checked and maybe there is a need to stop selling this kind of products.
I will continue the analysis and check the profit data of each market:

In [112]:
%%HTML
<div class='tableauPlaceholder' id='viz1693298572663' style='position: relative'><noscript><a href='#'><img alt='Sales &amp; Profit per Market ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ec&#47;EcommereceFullDataDashboards&#47;SalesProfitperMarket&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='EcommereceFullDataDashboards&#47;SalesProfitperMarket' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ec&#47;EcommereceFullDataDashboards&#47;SalesProfitperMarket&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1693298572663');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [111]:
market_df = df.groupby(['Market'])[['Sales','Profit']].sum().reset_index()
market_df['Profit Percentage'] = round((market_df['Profit']/market_df['Sales'])*100,2)
display(market_df.sort_values('Profit Percentage',ascending = False))


,Market,Sales,Profit,Profit Percentage
2,Europe,3281445,445907,13.59
4,USCA,2358033,300820,12.76
0,Africa,781499,87683,11.22
3,LATAM,2159583,219350,10.16
1,Asia Pacific,4035462,400876,9.93
